In [47]:
import fundamentalanalysis as fa
import pickle
from urllib.request import urlopen
import certifi
import json
import pandas as pd
from datetime import datetime

with open('/home/craigc/.keys/financialmodelingprep.txt', 'r') as file:
    api_key = file.read()

In [8]:
all_companies = fa.available_companies(api_key)
lse_companies = all_companies[(all_companies.exchangeShortName == 'LSE')]
print(f"num LSE companies: {len(lse_companies)}")
lse_stocks = lse_companies[(all_companies.type == 'stock')]
lse_stocks = lse_stocks.drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
print(f"num LSE stocks: {len(lse_stocks)}")
lse_stocks

num LSE companies: 6335
num LSE stocks: 3877


/tmp/ipykernel_109042/1136405498.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lse_stocks = lse_companies[(all_companies.type == 'stock')]


,name
symbol,
AIR3.L,Leverage Shares 3x Long Airbus (AIR.PA) ETC
KINO.L,Kinovo plc
HILS.L,Hill & Smith Holdings PLC
OSI.L,Osirium Technologies PLC
SRT.L,SRT Marine Systems plc
...,...
IDEA.L,Ideagen plc
MVI.L,Marwyn Value Investors Limited
0KD1.L,"Tubacex, S.A."


In [12]:
with open('../data/all_lse_stock_names_221219.pkl', 'wb') as file:
    pickle.dump(lse_stocks, file)

In [13]:
with open('../data/all_lse_stock_names_221219.pkl', 'rb') as file:
    lse_stocks = pickle.load(file)

print(type(lse_stocks))
print(len(lse_stocks))
print(lse_stocks)

<class 'pandas.core.frame.DataFrame'>
3877
                                               name
symbol                                             
AIR3.L  Leverage Shares 3x Long Airbus (AIR.PA) ETC
KINO.L                                   Kinovo plc
HILS.L                    Hill & Smith Holdings PLC
OSI.L                      Osirium Technologies PLC
SRT.L                        SRT Marine Systems plc
...                                             ...
IDEA.L                                  Ideagen plc
MVI.L                Marwyn Value Investors Limited
0KD1.L                                Tubacex, S.A.
DOCS.L                              Dr. Martens plc
BKY.L                      Berkeley Energia Limited

[3877 rows x 1 columns]


In [148]:
def fmp_request(api: str, ticker: str, api_key, v4: bool=False):
    '''
    Sends an API request to financialmodelingprep.com, converting the response to a df

        Parameters:
            api (str):     The API to query, e.g. 'income-statement'
            ticker (str):  The tickername to query, e.g 'MSFT'
            api_key (str): Your fmp API key

        Returns:
            dataframe
    '''
    if v4:
        url = f'https://financialmodelingprep.com/api/v4/{api}?symbol={ticker}&apikey={api_key}'
    else:
        url = f"https://financialmodelingprep.com/api/v3/{api}/{ticker}?apikey={api_key}"
    return pd.read_json(url)

In [221]:
def get_ticker_data(ticker: str, api_key: str):
    data = {}
    data['timestamp'] = datetime.timestamp(datetime.now())
    data['income_statement'] = fa.income_statement(ticker, api_key, period="annual").drop(labels=['link', 'finalLink'], axis=0)
    data['income_statement_growth'] = fmp_request('income-statement-growth', ticker, api_key).drop(['symbol', 'period'], axis=1)
    data['cashflow_statement'] = fa.cash_flow_statement(ticker, api_key, period="annual").drop(labels=['link', 'finalLink'], axis=0)
    data['cashflow_statement_growth'] = fmp_request('cash-flow-statement-growth', ticker, api_key).drop(['symbol', 'period'], axis=1)
    data['balance_sheet_statement'] = fa.balance_sheet_statement(ticker, api_key, period="annual").drop(labels=['link', 'finalLink'], axis=0)
    data['balance_sheet_statement_growth'] = fmp_request('balance-sheet-statement-growth', ticker, api_key).drop(['symbol', 'period'], axis=1)
    data['discounted_cash_flow'] = fmp_request('discounted-cash-flow', ticker, api_key).drop('symbol', axis=1)
    data['advanced_discounted_cash_flow'] = fmp_request('advanced_discounted_cash_flow', ticker, api_key, v4=True).drop(['symbol'], axis=1)
    data['advanced_levered_discounted_cash_flow'] = fmp_request('advanced_levered_discounted_cash_flow', ticker, api_key, v4=True).drop(['symbol'], axis=1)
    data['historical_discounted_cash_flow_statement'] = fmp_request('historical-discounted-cash-flow-statement', ticker, api_key).drop(['symbol'], axis=1)
    data['key_metrics'] = fa.key_metrics(ticker, api_key, period="annual")
    data['financial_ratios'] = fa.financial_ratios(ticker, api_key, period="annual")
    data['growth'] = fa.financial_statement_growth(ticker, api_key, period="annual")
    data['dividends'] = fa.stock_dividend(ticker, api_key).drop(['recordDate', 'declarationDate'], axis=1)
    data['scores'] = fmp_request('score', ticker, api_key, v4=True).drop(['symbol'], axis=1)
    data['enterprise_values'] = fmp_request('enterprise-values', ticker, api_key).drop(['symbol'], axis=1)

    return data


In [185]:
ticker = 'POLY.L'
display(fmp_request('discounted-cash-flow', ticker, api_key))
display(fmp_request('advanced_discounted_cash_flow', ticker, api_key, v4=True))
display(fmp_request('advanced_levered_discounted_cash_flow', ticker, api_key, v4=True))
display(fmp_request('historical-discounted-cash-flow-statement', ticker, api_key))

,symbol,date,dcf,Stock Price
0,POLY.L,2022-12-19,247.884826,247


,year,symbol,revenue,revenuePercentage,ebitda,ebitdaPercentage,ebit,ebitPercentage,depreciation,depreciationPercentage,...,ufcf,sumPvUfcf,longTermGrowthRate,terminalValue,presentTerminalValue,enterpriseValue,netDebt,equityValue,equityValuePerShare,freeCashFlowT1
0,2026,POLY.L,5.447790e+09,13.52,2.504060e+09,45.96,1.979260e+09,36.33,524800000.0,45.96,...,774050000.0,2502430000,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
1,2025,POLY.L,4.799050e+09,13.52,2.205880e+09,45.96,1.743570e+09,36.33,462310000.0,45.96,...,681880000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
2,2024,POLY.L,4.227570e+09,13.52,1.943200e+09,45.96,1.535940e+09,36.33,407260000.0,45.96,...,600680000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
3,2023,POLY.L,3.724150e+09,13.52,1.711800e+09,45.96,1.353040e+09,36.33,358760000.0,45.96,...,529150000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
4,2022,POLY.L,3.280670e+09,13.52,1.507950e+09,45.96,1.191910e+09,36.33,316040000.0,45.96,...,429980000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
5,2021,POLY.L,2.890000e+09,0.87,1.441000e+09,49.86,1.227000e+09,42.46,214000000.0,49.86,...,322390000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
6,2020,POLY.L,2.865000e+09,27.84,1.682000e+09,58.71,1.475000e+09,51.48,207000000.0,58.71,...,763110000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
7,2019,POLY.L,2.241000e+09,31.36,9.390000e+08,41.90,6.960000e+08,31.06,243000000.0,41.90,...,241580000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
8,2018,POLY.L,1.706000e+09,-6.01,6.770000e+08,39.68,4.910000e+08,28.78,186000000.0,39.68,...,247000000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
9,2017,POLY.L,1.815000e+09,0.00,7.200000e+08,39.67,5.060000e+08,27.88,214000000.0,39.67,...,235340000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000


,year,symbol,revenue,revenuePercentage,capitalExpenditure,capitalExpenditurePercentage,price,beta,dilutedSharesOutstanding,costofDebt,...,longTermGrowthRate,freeCashFlow,terminalValue,presentTerminalValue,enterpriseValue,netDebt,equityValue,equityValuePerShare,freeCashFlowT1,operatingCashFlowPercentage
0,2026,POLY.L,5.447790e+09,13.52,-1.169460e+09,-21.47,241,0.496,471280000.0,3.14,...,2,720370000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
1,2025,POLY.L,4.799050e+09,13.52,-1.030200e+09,-21.47,241,0.496,471280000.0,3.14,...,2,634590000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
2,2024,POLY.L,4.227570e+09,13.52,-9.075200e+08,-21.47,241,0.496,471280000.0,3.14,...,2,559020000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
3,2023,POLY.L,3.724150e+09,13.52,-7.994500e+08,-21.47,241,0.496,471280000.0,3.14,...,2,492450000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
4,2022,POLY.L,3.280670e+09,13.52,-7.042500e+08,-21.47,241,0.496,471280000.0,3.14,...,2,433810000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
5,2021,POLY.L,2.890000e+09,0.87,-7.590000e+08,-26.26,241,0.496,471280000.0,3.14,...,2,436000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,41.35
6,2020,POLY.L,2.865000e+09,27.84,-5.830000e+08,-20.35,241,0.496,471280000.0,3.14,...,2,609000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,41.61
7,2019,POLY.L,2.241000e+09,31.36,-4.360000e+08,-19.46,241,0.496,471280000.0,3.14,...,2,260000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,31.06
8,2018,POLY.L,1.706000e+09,-6.01,-3.440000e+08,-20.16,241,0.496,471280000.0,3.14,...,2,169000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,30.07
9,2017,POLY.L,1.815000e+09,0.00,-3.830000e+08,-21.10,241,0.496,471280000.0,3.14,...,2,150000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,29.37


,symbol,date,price,dcf
0,POLY.L,2021-12-31,1048.5,11.37
1,POLY.L,2020-12-31,1581.0,16.63
2,POLY.L,2019-12-31,1266.0,13.20
3,POLY.L,2018-12-31,857.6,9.42
4,POLY.L,2017-12-31,825.8,8.34
5,POLY.L,2016-12-31,898.0,9.09
6,POLY.L,2015-12-31,575.0,5.87
7,POLY.L,2014-12-31,597.5,6.37
8,POLY.L,2013-12-31,579.5,5.96
9,POLY.L,2012-12-31,1088.0,10.93


In [223]:
ticker = 'POLY.L'
data = get_ticker_data(ticker, api_key)
display(data)

{'timestamp': 1671467989.681481,
 'income_statement':                                                2021        2020        2019  \
 reportedCurrency                                USD         USD         USD   
 cik                                            None        None        None   
 fillingDate                              2021-12-31  2020-12-31  2019-12-31   
 acceptedDate                             2021-12-31  2020-12-31  2019-12-31   
 calendarYear                                   2021        2020        2019   
 period                                           FY          FY          FY   
 revenue                                  2890000000  2865000000  2241000000   
 costOfRevenue                            1307000000  1129000000  1197000000   
 grossProfit                              1583000000  1736000000  1044000000   
 grossProfitRatio                           0.547751    0.605934    0.465863   
 researchAndDevelopmentExpenses                  0.0         0.0   

In [191]:
print(data.keys())
key = 'growth'
display(data[key].keys())
display(data[key])
# display(data[key].drop(labels=['link', 'finalLink'], axis=0))

dict_keys(['timestamp', 'income_statement', 'income_statement_growth', 'cashflow_statement', 'cashflow_statement_growth', 'balance_sheet_statement', 'balance_sheet_statement_growth', 'discounted_cash_flow', 'advanced_discounted_cash_flow', 'advanced_levered_discounted_cash_flow', 'historical_discounted_cash_flow_statement', 'key_metrics', 'financial_ratios', 'growth', 'dividends', 'scores', 'enterprise_values'])


Index(['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013',
       '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005'],
      dtype='object')

,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
revenueGrowth,0.008726,0.278447,0.313599,-0.060055,0.146557,0.098472,-0.147479,-0.009496,-0.079538,0.397785,0.433397,0.650285,0.115382,0.628294,-0.021702,0.320635,0.0
grossProfitGrowth,-0.088134,0.662835,0.420408,0.036671,-0.037992,0.11352,-0.062973,0.644677,-0.554719,0.376499,0.49957,0.691009,0.367912,2.733242,-0.625058,0.403823,0.0
ebitgrowth,-0.167123,0.836478,0.517176,0.031496,-0.127148,0.201476,0.072488,1.476785,-0.714315,0.306826,0.473366,0.782622,1.317346,10.364411,-1.089184,0.361954,0.0
operatingIncomeGrowth,-0.167123,0.836478,0.517176,0.031496,-0.127148,0.201476,0.072488,1.476785,-0.714315,0.306826,0.473366,0.782622,1.317346,10.364411,-1.089184,0.361954,0.0
netIncomeGrowth,-0.167587,1.2625,0.352113,0.002825,-0.103797,0.787371,2.05316,-0.0596,-1.502188,0.362999,0.210483,1.490052,7.102225,0.310874,-1.370029,2.505142,0.0
epsgrowth,-0.166007,1.254902,0.289507,-0.035366,-0.11828,0.788462,1.981132,-0.039216,-1.495146,0.303797,0.135057,1.33557,6.96,0.324324,-1.330357,2.5,0.0
epsdilutedGrowth,-0.154985,1.247525,0.276865,-0.023457,-0.129032,0.788462,1.981132,-0.039216,-1.495146,0.391892,0.072464,1.37931,6.8,0.324324,-1.330357,2.5,0.0
weightedAverageSharesGrowth,0.0,0.002879,0.046567,0.044515,0.00879,0.00751,0.068281,0.019615,0.014638,0.042883,0.022961,0.112888,0.031662,0.015831,0.118478,0.0,0.0
weightedAverageSharesDilutedGrowth,-0.014035,0.003329,0.057463,0.033973,0.021851,0.007432,0.069015,0.019615,0.014638,-0.02581,0.085591,0.093184,0.05945,0.015831,0.118478,0.0,0.0


In [231]:
import sys
import math

def get_mem_usage_kb(data: dict):

    mem_usage_bytes = 0
    for val in data.values():
        if isinstance(val, pd.DataFrame):
            mem_usage_bytes += val.memory_usage().sum()
    
    return math.ceil(mem_usage_bytes / 1024)

data_mem_kb = get_mem_usage_kb(data)
print(f"Data is using {get_mem_usage_kb(data)}kb")
print(f"Total memory use: {len(lse_stocks) * data_mem_kb / 1024}")

request_duration = 10.6
print(f"Total download time: {len(lse_stocks) * request_duration / 3600: .1f}h")

Data is using 55kb
Total memory use: 208.2373046875
Total download time:  11.4h


In [201]:
for key, val in data.items():
    print(key)
    display(val)

timestamp
income_statement
income_statement_growth
cashflow_statement
cashflow_statement_growth
balance_sheet_statement
balance_sheet_statement_growth
discounted_cash_flow
advanced_discounted_cash_flow
advanced_levered_discounted_cash_flow
historical_discounted_cash_flow_statement
key_metrics
financial_ratios
growth
dividends
scores
enterprise_values


In [230]:
print('timestamp')
display(data['timestamp'])
print('income_statement')
display(data['income_statement'])
print('income_statement_growth')
display(data['income_statement_growth']) # rotate
print('cashflow_statement')
display(data['cashflow_statement'])
print('cashflow_statement_growth')
display(data['cashflow_statement_growth']) # rotate
print('balance_sheet_statement')
display(data['balance_sheet_statement'])
print('balance_sheet_statement_growth')
display(data['balance_sheet_statement_growth']) # rotate
print('discounted_cash_flow')
display(data['discounted_cash_flow'])
print('advanced_discounted_cash_flow')
display(data['advanced_discounted_cash_flow']) # rotate
print('advanced_levered_discounted_cash_flow')
display(data['advanced_levered_discounted_cash_flow']) # rotate
print('historical_discounted_cash_flow_statement')
display(data['historical_discounted_cash_flow_statement']) # rotate
print('key_metrics')
display(data['key_metrics'])
print('financial_ratios')
display(data['financial_ratios'])
print('growth')
display(data['growth'])
print('dividends')
display(data['dividends']) # aggregate & rotate?
print('scores')
display(data['scores'])
print('enterprise_values')
display(data['enterprise_values']) # rotate

timestamp


1671467989.681481

income_statement


,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
cik,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
fillingDate,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31,2010-12-31,2009-12-31,2008-12-31,2007-12-31,2006-12-31,2005-12-31
acceptedDate,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31,2010-12-31,2009-12-31,2008-12-31,2007-12-31,2006-12-31,2005-12-31
calendarYear,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
revenue,2890000000,2865000000,2241000000,1706000000,1815000000,1583000000,1441093000,1690391000,1706597000,1854065000,1326430566.4,925375671,560737000,502731000,308747000,315596000,238973000
costOfRevenue,1307000000,1129000000,1197000000,971000000,1106000000,846000000,779228000,984045000,1277123000,889565000,625739715.8,458114425.8,284416000,300729000,254638000,171283000,136173000
grossProfit,1583000000,1736000000,1044000000,735000000,709000000,737000000,661865000,706346000,429474000,964500000,700690850.6,467261245.2,276321000,202002000,54109000,144313000,102800000
grossProfitRatio,0.547751,0.605934,0.465863,0.430832,0.390634,0.465572,0.45928,0.41786,0.251655,0.520208,0.528253,0.504942,0.492782,0.401809,0.175254,0.457271,0.430174


income_statement_growth


...

,label,adjDividend,dividend,paymentDate
2022-05-05,"May 05, 22",52.0,52.0,
2021-09-09,"September 09, 21",32.587,32.587,
2021-05-06,"May 06, 21",63.3537,63.3537,
2020-09-03,"September 03, 20",30.9198,30.9198,
2020-05-07,"May 07, 20",34.397,0.42,2020-05-29
2020-02-13,"February 13, 20",15.3908,0.2,2020-03-05
2019-09-05,"September 05, 19",16.1696,0.2,2019-09-27
2019-05-09,"May 09, 19",23.974,0.31,2019-05-24
2018-09-06,"September 06, 18",13.068,0.17,2018-09-28
2018-05-10,"May 10, 18",22.2585,0.3,2018-05-23


scores


,altmanZScore,piotroskiScore,workingCapital,totalAssets,retainedEarnings,ebit,marketCap,totalLiabilities,revenue
0,1.160701,7,1188000000,7436000000,1252000000,644750000,1136698752,4165000000,2107250000


enterprise_values


,date,stockPrice,numberOfShares,marketCapitalization,minusCashAndCashEquivalents,addTotalDebt,enterpriseValue
0,2021-12-31,1048.500000,471278987,4.941360e+11,417000000.0,2.100000e+09,6.624360e+09
1,2020-12-31,1581.000000,471278987,7.450921e+11,386000000.0,1.770000e+09,8.834921e+09
2,2019-12-31,1266.000000,469926157,5.949265e+11,253000000.0,1.764000e+09,7.460265e+09
3,2018-12-31,857.599976,449016966,3.850769e+11,379000000.0,1.899000e+09,5.370769e+09
4,2017-12-31,825.799988,429880907,3.549956e+11,36000000.0,1.456000e+09,4.969956e+09
5,2016-12-31,898.000000,426135182,3.826694e+11,48000000.0,1.378000e+09,5.156694e+09
6,2015-12-31,575.000000,422958680,2.432012e+11,51798000.0,1.349546e+09,3.729760e+09
7,2014-12-31,597.500000,395924528,2.365649e+11,157224000.0,1.322635e+09,3.531060e+09
8,2013-12-31,579.500000,388307843,2.250244e+11,65567000.0,1.111144e+09,3.295821e+09
9,2012-12-31,1088.000000,382705692,4.163838e+11,18622000.0,8.638230e+08,5.009039e+09


## TODO:
### Format:
- enterprise_values
- income_statement_growth
- cashflow_statement_growth
- balance_sheet_statement_growth
- advanced_discounted_cash_flow
- advanced_levered_discounted_cash_flow
- historical_discounted_cash_flow_statement
- dividends
- enterprise_values